In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
from sklearn import svm, ensemble
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# 1. Data Loaders
# MNIST for Model1 and Model2
tf_mnist = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_mnist = datasets.MNIST(root='./data', train=True, download=True, transform=tf_mnist)
test_mnist = datasets.MNIST(root='./data', train=False, download=True, transform=tf_mnist)
loader_train_mnist = DataLoader(train_mnist, batch_size=64, shuffle=True)
loader_test_mnist = DataLoader(test_mnist, batch_size=1000, shuffle=False)

In [4]:
# CIFAR-10 for Model3
# Resize to 224x224 for pretrained models
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]
tf_cifar = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])
train_cifar = datasets.CIFAR10(root='./data', train=True, download=True, transform=tf_cifar)
test_cifar = datasets.CIFAR10(root='./data', train=False, download=True, transform=tf_cifar)
loader_train_cifar = DataLoader(train_cifar, batch_size=64, shuffle=True)
loader_test_cifar = DataLoader(test_cifar, batch_size=1000, shuffle=False)

In [5]:
# 2. Model 1: LeNet-5-like CNN
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16*4*4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [6]:
# 3. Model 2: Improved LeNet with BatchNorm and Dropout
class ImprovedLeNet(nn.Module):
    def __init__(self):
        super(ImprovedLeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16*4*4, 120),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(84, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [7]:
# Utility: Training and Evaluation

def train_model(model, loader_train, criterion, optimizer, num_epochs=10):
    model = model.to(device)
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in loader_train:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(loader_train.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    return model

def evaluate_model(model, loader_test):
    model = model.to(device)
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in loader_test:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    acc = accuracy_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)
    print(f'Accuracy: {acc:.4f}')
    print('Confusion Matrix:\n', cm)
    return acc, cm


In [8]:
# 4. Model 3: Pretrained VGG11 on CIFAR-10
vgg11 = models.vgg11(pretrained=True)
vgg11.classifier[6] = nn.Linear(4096, 10)

C:\Users\berat\.julia\conda\3\x86_64\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\berat\.julia\conda\3\x86_64\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to C:\Users\berat/.cache\torch\hub\checkpoints\vgg11-8a719046.pth


100%|███████████████████████████████████████████████████████████████████████████████| 507M/507M [06:48<00:00, 1.30MB/s]


In [9]:
# 5. Model 4: Feature Extraction + SVM

def extract_features(model, loader):
    model = model.to(device)
    model.eval()
    feats, labs = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            x = model.features(images)
            x = torch.flatten(x, 1)
            feats.append(x.cpu().numpy())
            labs.append(labels.numpy())
    return np.concatenate(feats), np.concatenate(labs)

In [ ]:
# 6. Main Experiment
if __name__ == '__main__':
    # LeNet
    lenet = LeNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(lenet.parameters(), lr=0.001)
    print('Training LeNet...')
    train_model(lenet, loader_train_mnist, criterion, optimizer, num_epochs=10)
    print('Evaluating LeNet...')
    evaluate_model(lenet, loader_test_mnist)

    # Improved LeNet
    imp_lenet = ImprovedLeNet()
    optimizer2 = optim.Adam(imp_lenet.parameters(), lr=0.001)
    print('Training Improved LeNet...')
    train_model(imp_lenet, loader_train_mnist, criterion, optimizer2, num_epochs=10)
    print('Evaluating Improved LeNet...')
    evaluate_model(imp_lenet, loader_test_mnist)

    # Pretrained VGG11
    vgg = vgg11.to(device)
    opt_vgg = optim.Adam(vgg.parameters(), lr=0.0001)
    print('Training Pretrained VGG11 on CIFAR-10...')
    train_model(vgg, loader_train_cifar, criterion, opt_vgg, num_epochs=5)
    print('Evaluating VGG11...')
    evaluate_model(vgg, loader_test_cifar)

    # Features + SVM (using LeNet)
    print('Extracting features from LeNet...')
    X_train, y_train = extract_features(lenet, loader_train_mnist)
    X_test, y_test = extract_features(lenet, loader_test_mnist)
    np.save('features_train.npy', X_train)
    np.save('labels_train.npy', y_train)
    np.save('features_test.npy', X_test)
    np.save('labels_test.npy', y_test)
    print('Training SVM on extracted features...')
    svc = svm.SVC()
    svc.fit(X_train, y_train)
    y_pred_svm = svc.predict(X_test)
    print('SVM Accuracy:', accuracy_score(y_test, y_pred_svm))

    # Full CNN comparison complete
    print('All experiments completed.')


Training LeNet...
Epoch [1/10], Loss: 0.2356
Epoch [2/10], Loss: 0.0697
Epoch [3/10], Loss: 0.0506
Epoch [4/10], Loss: 0.0394
Epoch [5/10], Loss: 0.0324
Epoch [6/10], Loss: 0.0269
Epoch [7/10], Loss: 0.0238
Epoch [8/10], Loss: 0.0199
Epoch [9/10], Loss: 0.0174
Epoch [10/10], Loss: 0.0155
Evaluating LeNet...
Accuracy: 0.9856
Confusion Matrix:
 [[ 977    1    0    0    0    0    1    0    0    1]
 [   1 1117    0    3    0    2    8    2    2    0]
 [   0    1 1019    7    0    0    1    3    1    0]
 [   1    1    0 1005    0    1    0    0    0    2]
 [   0    0    0    0  965    0    6    0    2    9]
 [   0    0    0   13    0  871    3    0    0    5]
 [   4    1    0    0    1    2  950    0    0    0]
 [   0    6    4    1    1    1    0  996    1   18]
 [   2    0    2    2    0    0    1    0  959    8]
 [   1    2    0    1    6    1    0    1    0  997]]
Training Improved LeNet...
Epoch [1/10], Loss: 0.3634
Epoch [2/10], Loss: 0.1237
Epoch [3/10], Loss: 0.1031
Epoch [4/10], Lo